# 初始化設定

## 將程式下載到colab環境

In [1]:
!git clone https://github.com/researchmm/AOT-GAN-for-Inpainting.git
%cd AOT-GAN-for-Inpainting/

Cloning into 'AOT-GAN-for-Inpainting'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 111 (delta 15), reused 8 (delta 8), pack-reused 79 (from 1)
Receiving objects: 100% (111/111), 25.39 MiB | 30.81 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/AOT-GAN-for-Inpainting


## 上傳資料集

In [2]:
#將資料集複製到執行目錄下(由於dataset.py的載入方法基於執行目錄的相對路徑，因此在不更動程式的情況下這樣處理)
import shutil

In [ ]:
# 複製data資料集
source_path = "/content/drive/MyDrive/中興_深度學習/HW4/data"  # 請替換成你的data資料集的絕對路徑
target_path = "/content/AOT-GAN-for-Inpainting/mydata"        # Colab 中的執行目錄
shutil.copytree(source_path, target_path)
print(f"資料夾已從 {source_path} 複製到 {target_path}")


資料夾已從 /content/drive/MyDrive/中興_深度學習/HW4/data 複製到 /content/AOT-GAN-for-Inpainting/mydata


In [5]:
# 複製test資料集
source_path = "/content/drive/MyDrive/中興_深度學習/HW4/test"  # 請替換成你的test資料集的絕對路徑
target_path = "/content/AOT-GAN-for-Inpainting/mytest"        # Colab 中的執行目錄
shutil.copytree(source_path, target_path)
print(f"資料夾已從 {source_path} 複製到 {target_path}")

資料夾已從 /content/drive/MyDrive/中興_深度學習/HW4/test 複製到 /content/AOT-GAN-for-Inpainting/mytest


# Train place2

## 設定參數

In [ ]:
DIR_IMG = '/content/AOT-GAN-for-Inpainting/mydata'
DIR_MASK = '/content/AOT-GAN-for-Inpainting/mydata'
DATA_TRAIN = 'place2_train'
MASK_TYPE = 'train_mask'

MAX_ITER = 5000 # --iterations
nthread = 2 # --num_workers
SAVE_FREQ = 500 # --save_every
Batch_SIZE = 4 # batch_size
IMG_SIZE = 256 # --image_size

In [ ]:
# 請以下方程式替換src.trainer.trainer.py的第147行，確保訓練結束時至少保存一次
if self.args.global_rank == 0 and ((self.iteration % self.args.save_every) == 0 or (self.iteration == self.args.iterations)):


#請以下方程式替換src.trainer.trainer.py的第73行整個函數，確保儲存邏輯無誤
def save(self,):
        if self.args.global_rank == 0:
            print(f"\nsaving {self.iteration} model to {self.args.save_dir} ...")
            if (self.args.distributed == True):
                model_to_save_G = self.netG.module
                model_to_save_D = self.netD.module
            else:
                model_to_save_G = self.netG
                model_to_save_D = self.netD

            torch.save(
                model_to_save_G.state_dict(), os.path.join(self.args.save_dir, f"G{str(self.iteration).zfill(7)}.pt")
            )
            torch.save(
                model_to_save_D.state_dict(), os.path.join(self.args.save_dir, f"D{str(self.iteration).zfill(7)}.pt")
            )
            torch.save(
                {"optimG": self.optimG.state_dict(), "optimD": self.optimD.state_dict()},
                os.path.join(self.args.save_dir, f"O{str(self.iteration).zfill(7)}.pt"),
            )

## 開始訓練

In [ ]:
!python src/train.py --dir_image $DIR_IMG --data_train $DATA_TRAIN --dir_mask $DIR_MASK --mask_type $MASK_TYPE --image_size 256 --iterations $MAX_ITER --num_workers $nthread --save_every $SAVE_FREQ --batch_size $Batch_SIZE

2024-12-31 01:32:43.249508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-31 01:32:43.270094: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-31 01:32:43.275934: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-31 01:32:43.294375: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-31 01:32:44.651680: W tensorflow/comp

## 回復訓練狀態

In [ ]:
#將來源路徑替換成你的儲存路徑
!cp -r /content/drive/MyDrive/中興_深度學習/HW4/4-AOT-GAN/place2/experiments /content

## 保存訓練後的模型

In [ ]:
#將目標路徑替換成你的儲存路徑
!cp -r /content/experiments /content/drive/MyDrive/中興_深度學習/HW4/4-AOT-GAN/place2

# Test place2

## 設定參數

In [ ]:
test_data_names = ["FFHQ_test", "Shunghaitech_test", "celeba_test", "paris_test", "place2_chruch_indoor_test", "place2_chruch_ourdoor_test"]
test_mask_name="test_mask"

dir_root = "/content/AOT-GAN-for-Inpainting/mytest"
dir_output = "/content/output"

In [ ]:
PRE_TRAIN = '/content/drive/MyDrive/中興_深度學習/HW4/4-AOT-GAN/place2/experiments/aotgan_place2_train_train_mask256/G0005000.pt' # --pre_train

nthread = 2 # --num_workers
Batch_SIZE = 4 # batch_size
IMG_SIZE = 256 # --image_size

## 開始測試

In [ ]:
test_data_names = ["FFHQ_test", "Shunghaitech_test", "celeba_test", "paris_test", "place2_chruch_indoor_test", "place2_chruch_ourdoor_test"]
test_mask_name="test_mask"

dir_root = "/content/AOT-GAN-for-Inpainting/mytest"
dir_output = "/content/output"

PRE_TRAIN = '/content/drive/MyDrive/中興_深度學習/HW4/4-AOT-GAN/place2/experiments/aotgan_place2_train_train_mask256/G0005000.pt' # --pre_train

nthread = 2 # --num_workers
Batch_SIZE = 4 # batch_size
IMG_SIZE = 256 # --image_size

In [ ]:
for data_test in test_data_names:
  print("Processing: ", data_test)
  DIR_IMG = f"{dir_root}/{data_test}"
  DIR_MASK = f"{dir_root}/{test_mask_name}"
  OUTPUTS = f"{dir_output}/{data_test}"
  !python src/test.py --dir_image $DIR_IMG --dir_mask $DIR_MASK --outputs $OUTPUTS --pre_train $PRE_TRAIN --num_workers $nthread --batch_size $Batch_SIZE --image_size $IMG_SIZE

Processing:  FFHQ_test
/content/AOT-GAN-for-Inpainting/src/test.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(args.pre_train, map_loc

## 保存測試結果

In [ ]:
!cp -r /content/output /content/drive/MyDrive/中興_深度學習/HW4/4-AOT-GAN/place2

# Train celeba

## 設定參數

In [ ]:
DIR_IMG = '/content/AOT-GAN-for-Inpainting/mydata'
DIR_MASK = '/content/AOT-GAN-for-Inpainting/mydata'
DATA_TRAIN = 'celeba_train'
MASK_TYPE = 'train_mask'

MAX_ITER = 5000 # --iterations
nthread = 2 # --num_workers
SAVE_FREQ = 500 # --save_every
Batch_SIZE = 4 # batch_size

In [ ]:
# 請以下方程式替換src.trainer.trainer.py的第147行，確保訓練結束時至少保存一次
if self.args.global_rank == 0 and ((self.iteration % self.args.save_every) == 0 or (self.iteration == self.args.iterations)):


#請以下方程式替換src.trainer.trainer.py的第73行整個函數，確保儲存邏輯無誤
def save(self,):
        if self.args.global_rank == 0:
            print(f"\nsaving {self.iteration} model to {self.args.save_dir} ...")
            if (self.args.distributed == True):
                model_to_save_G = self.netG.module
                model_to_save_D = self.netD.module
            else:
                model_to_save_G = self.netG
                model_to_save_D = self.netD

            torch.save(
                model_to_save_G.state_dict(), os.path.join(self.args.save_dir, f"G{str(self.iteration).zfill(7)}.pt")
            )
            torch.save(
                model_to_save_D.state_dict(), os.path.join(self.args.save_dir, f"D{str(self.iteration).zfill(7)}.pt")
            )
            torch.save(
                {"optimG": self.optimG.state_dict(), "optimD": self.optimD.state_dict()},
                os.path.join(self.args.save_dir, f"O{str(self.iteration).zfill(7)}.pt"),
            )

## 開始訓練

In [ ]:
!python src/train.py --dir_image $DIR_IMG --data_train $DATA_TRAIN --dir_mask $DIR_MASK --mask_type $MASK_TYPE --image_size 256 --iterations $MAX_ITER --num_workers $nthread --save_every $SAVE_FREQ --batch_size $Batch_SIZE

2024-12-31 02:37:59.382209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-31 02:37:59.403497: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-31 02:37:59.411944: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-31 02:37:59.428631: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-31 02:38:00.761615: W tensorflow/comp

## 回復訓練狀態

In [ ]:
#將來源路徑替換成你的儲存路徑
!cp -r /content/drive/MyDrive/中興_深度學習/HW4/4-AOT-GAN/celeba/experiments /content

## 保存訓練後的模型

In [ ]:
#將目標路徑替換成你的儲存路徑
!cp -r /content/experiments /content/drive/MyDrive/中興_深度學習/HW4/4-AOT-GAN/celeba

# Test celeba

## 設定參數

In [ ]:
test_data_names = ["FFHQ_test", "Shunghaitech_test", "celeba_test", "paris_test", "place2_chruch_indoor_test", "place2_chruch_ourdoor_test"]
#test_data_names = ["place2_chruch_ourdoor_test"]
test_mask_name="test_mask"

dir_root = "/content/AOT-GAN-for-Inpainting/mytest"
dir_output = "/content/output"

In [ ]:
PRE_TRAIN = '/content/drive/MyDrive/中興_深度學習/HW4/4-AOT-GAN/celeba/experiments/aotgan_celeba_train_train_mask256/G0005000.pt' # --pre_train
nthread = 2 # --num_workers
Batch_SIZE = 4 # batch_size
IMG_SIZE = 256 # --image_size

## 開始測試

In [ ]:
for data_test in test_data_names:
  print("Processing: ", data_test)
  DIR_IMG = f"{dir_root}/{data_test}"
  DIR_MASK = f"{dir_root}/{test_mask_name}"
  OUTPUTS = f"{dir_output}/{data_test}"
  !python src/test.py --dir_image $DIR_IMG --dir_mask $DIR_MASK --outputs $OUTPUTS --pre_train $PRE_TRAIN --num_workers $nthread --batch_size $Batch_SIZE --image_size $IMG_SIZE

Processing:  place2_chruch_ourdoor_test
/content/AOT-GAN-for-Inpainting/src/test.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(args.p

## 保存測試結果

In [ ]:
!cp -r /content/output /content/drive/MyDrive/中興_深度學習/HW4/4-AOT-GAN/celeba

# Complexity

## 設定參數

In [6]:
test_data_names = ["FFHQ_test", "Shunghaitech_test", "celeba_test", "paris_test", "place2_chruch_indoor_test", "place2_chruch_ourdoor_test"]
test_mask_name="test_mask"

dir_root = "/content/AOT-GAN-for-Inpainting/mytest"
dir_output = "/content/output"

In [7]:
PRE_TRAIN = '/content/drive/MyDrive/中興_深度學習/HW4/4-AOT-GAN/celeba/experiments/aotgan_celeba_train_train_mask256/G0005000.pt' # --pre_train
nthread = 2 # --num_workers
Batch_SIZE = 4 # batch_size
IMG_SIZE = 256 # --image_size

In [9]:
!pip install ptflops

In [ ]:
# 在test.py 第27行後添加以下程式
    # Dummy input for FLOPs and parameter calculation
    device = torch.device("cuda") if use_gpu else torch.device("cpu")
    dummy_input = torch.randn(1, 2, args.image_size, args.image_size).to(device)

    # Calculate parameters and FLOPs
    from ptflops import get_model_complexity_info
    with torch.no_grad():
        try:
            flops, params = get_model_complexity_info(
                model,
                (2, args.image_size, args.image_size),
                as_strings=True,
                print_per_layer_stat=False,
                verbose=True,
            )
            print(f"Model FLOPs: {flops}")
            print(f"Model Parameters: {params}")
        except Exception as e:
            print(f"Error calculating FLOPs: {e}")


In [ ]:
# 在model/aotgan.py 第31行開始更改部分程式

def forward(self, x, mask=None):
    if mask is None:
      # 提供一個默認的 input_mask，形狀與 input_tensor 匹配
      mask = torch.ones_like(x[:, :, :, :])


## 開始測試

In [15]:
data_test = 'FFHQ_test'
print("Processing: ", data_test)
DIR_IMG = f"{dir_root}/{data_test}"
DIR_MASK = f"{dir_root}/{test_mask_name}"
OUTPUTS = f"{dir_output}/{data_test}"
!python src/test.py --dir_image $DIR_IMG --dir_mask $DIR_MASK --outputs $OUTPUTS --pre_train $PRE_TRAIN --num_workers $nthread --batch_size $Batch_SIZE --image_size $IMG_SIZE

Processing:  FFHQ_test
/content/AOT-GAN-for-Inpainting/src/test.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(args.pre_train, map_loc